In [2]:
import pennylane as qml
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from math import log, ceil
from tqdm import tqdm
import torch.multiprocessing as mp
import numpy as np

In [3]:
dataset = Planetoid("/tmp/Cora", "Cora", split="random", num_train_per_class=int(140/7))
(
    dataset.train_mask.sum().item(),
    dataset.val_mask.sum().item(),
    dataset.test_mask.sum().item()
)

(140, 500, 1000)

In [4]:
n = ceil(log(dataset.num_features)/log(2))
n

11

In [5]:
n_out = dataset.num_classes
n_out

7

In [6]:
dev = qml.device("default.qubit", wires=n)

In [7]:
@qml.qnode(dev, interface="torch")
def circuit(inputs, weights):
    qml.AmplitudeEmbedding(inputs, wires=range(n), pad_with=0.001, normalize=True)
    qml.StronglyEntanglingLayers(weights=weights, wires=range(n))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_out)]

In [8]:
# def single(layer, x):
#     return layer(x)

class QNNModel(torch.nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        
        self.q_layer_1 = qml.qnn.TorchLayer(circuit, {
            "weights": (n_layers, n, 3)
        })

    def single(self, x):
        return self.q_layer_1(x)
    
    def forward(self, x):
        out = torch.zeros(x.shape[0], n_out)

        for i in tqdm(range(x.shape[0])):
            out[i] = self.q_layer_1(x[i])
        
        # with mp.Pool() as p:
        #     res = [p.apply_async(single, args=(self.q_layer_1, x)) for x in data.x]
        #     for i in tqdm(range(data.x.shape[0])):
        #         out[i] = res[i].get()
            
        return out

In [15]:
class ClassicModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer = torch.nn.Linear(dataset.num_features, dataset.num_classes)

    def forward(self, x):
        x = self.layer(x)
        return x

In [ ]:
device = "cpu" # torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = QNNModel(3).to(device)
# model = ClassicModel().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

losses = []

model.train()
for epoch in range(200):
    print("Epoch", epoch)
    optimizer.zero_grad()
    out = model(data.x[data.train_mask])
    loss = F.nll_loss(out, data.y[data.train_mask])
    loss.backward()
    losses.append(loss)
    optimizer.step()

Epoch 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:08<00:00, 17.44it/s]


Epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:07<00:00, 17.52it/s]


Epoch 2


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [01:46<00:00,  1.31it/s]


Epoch 150


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [01:44<00:00,  1.34it/s]


Epoch 151


 14%|███████████████▎                                                                                                 | 19/140 [00:02<00:21,  5.57it/s]

In [36]:
model.eval()
pred = model(data.x[data.test_mask]).argmax(dim=1)
correct = (pred == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:31<00:00, 10.90it/s]


Accuracy: 0.1700


In [ ]:
import matplotlib.pyplot as plt
plt.